<a href="https://colab.research.google.com/github/Serendipity-Song/Classify_CIFAR-10_Dataset/blob/main/Cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# 데이터를 로드하고 전처리
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:04<00:00, 42563817.46it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)
model.eval()

# 양자화 설정
qconfig = torch.quantization.get_default_qconfig('fbgemm')
model.qconfig = qconfig

# 양자화 준비
model = torch.quantization.prepare(model, inplace=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 112MB/s]
/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_ra

In [3]:
for i, (inputs, labels) in enumerate(trainloader, 0):
    model(inputs)


In [4]:
model = torch.quantization.convert(model, inplace=True)


In [6]:
import tensorflow as tf

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()


170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalization
X = x_train.astype(np.float32) / 255.0

# Define and compile original model
base_model = tf.keras.applications.InceptionV3(include_top=False, input_shape=(96, 96, 3), weights=None)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model on CIFAR-10 dataset
X_resized = tf.image.resize(X, (96, 96))
model.fit(X_resized, y_train, epochs=5)

# Generate inputs for DDV computation
def generate_inputs(model, X, N=100, epsilon=0.01, lambda_=1.0):
    X_prime = X.copy()
    for i in range(N):
        indices = np.random.choice(X.shape[0], size=int(X.shape[0]*0.1), replace=False)
        pos = np.random.choice(np.prod(X.shape[1:]), size=1)
        X_prime_left = X_prime.copy()
        X_prime_right = X_prime.copy()
        X_prime_left[indices, pos//X.shape[2], pos%X.shape[2]] -= epsilon
        X_prime_right[indices, pos//X.shape[2], pos%X.shape[2]] += epsilon
        score_left = np.abs(model(X_resized) - model(tf.image.resize(X_prime_left, (96, 96)))).sum() + lambda_ * np.abs(X_prime_left - X).sum()
        score_right = np.abs(model(X_resized) - model(tf.image.resize(X_prime_right, (96, 96)))).sum() + lambda_ * np.abs(X_prime_right - X).sum()
        if score_left > score_right:
            X_prime = X_prime_left
        else:
            X_prime = X_prime_right
    return X_prime

# Compute DDV
def compute_ddv(model, X, X_prime):
    return np.abs(model(X).numpy() - model(X_prime).numpy()).sum(axis=1)

# Compute model similarity
def compute_similarity(DDV_f, DDV_g):
    return cosine_similarity([DDV_f], [DDV_g])[0][0]

# Generate inputs
X_prime = generate_inputs(model, X)

# Compute DDVs
DDV_f = compute_ddv(model, X_resized, tf.image.resize(X_prime, (96, 96)))
DDV_g = compute_ddv(model, X_resized, tf.image.resize(X_prime, (96, 96)))

# Compute model similarity
similarity = compute_similarity(DDV_f, DDV_g)
print("Model similarity:", similarity)


Epoch 1/5
 563/1563 [=========>....................] - ETA: 51:19 - loss: 1.9552 - accuracy: 0.3166